
L'obiettivo del progetto è apprendere mediante una rete neurale la trasformazione da punti del piano espressi in coordinate polari ad una rappresentazione basata su di una griglia discreta di dimensione 10x10, dove la cella della griglia ha valore 1 se contiene il punto, e 0 altrimenti.

Il dataset supervisionato è fornito in questo notebook nella forma di una generatore. Il generatore deve essere considerato come una "scatola nera" il cui comportamento deve essere appreso. 

Dovete progettare una rete neurale in grado di raggiungere una accuratezza del 95%. Questa è una condizione necessaria per superare l'esame, ma l'accuratezza non influisce in altro modo sulla valutazione.  

I modelli che raggiungono l'accuratezza attesa saranno invece valutati in modo inversamente proporzionale al numero dei loro parametri: **più il modello è piccolo, meglio è.**


**Attenzione**: Qualunque soluzione che tragga vantaggio, diretto o indiretto, da meta-conoscenza relativa al generatore sarà automaticamente bocciato.


In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Reshape, concatenate
from tensorflow.keras.models import Model
import keras
from keras import layers
from keras import activations
from sklearn.model_selection import train_test_split

Veniamo al generatore. Questo restituisce delle triple della forma
((theta,rho),out) dove (theta,rho) sono le coordinate polari di un punto nel primo quadrante del piano, e out è una mappa 10x10 con "1" in correspondenza alla cella che contiene il punto, e "0" altrimenti.

Settando  flat=True, la mappa 10x10 viene appiattita ad un vettore di dimensione 100. Potete utilizzare questa variante, se preferite. Nessuna altra modifica del generatore è ammessa. 

In [ ]:
def polar_generator(batchsize,grid=(10,10),noise=.002,flat=False):
  while True:
    x = np.random.rand(batchsize)
    y = np.random.rand(batchsize)
    out = np.zeros((batchsize,grid[0],grid[1]))
    xc = (x*grid[0]).astype(int)
    yc = (y*grid[1]).astype(int)
    for b in range(batchsize):
      out[b,xc[b],yc[b]] = 1
    #compute rho and theta and add some noise
    rho = np.sqrt(x**2+y**2) + np.random.normal(scale=noise)
    theta = np.arctan(y/np.maximum(x,.00001)) + np.random.normal(scale=noise)
    if flat:
      out = np.reshape(out,(batchsize,grid[0]*grid[1]))
    yield ((theta,rho),out)

In [ ]:
batch_size = 1024
n_test = 20000
n_train = 4000000
g1,g2 = 10,10
generate = polar_generator(n_train + n_test, grid=(g1,g2), noise=0.002, flat=True)

In [ ]:
(theta,rho),y = next(generate)
x = np.array(list(zip(theta, rho)))

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=n_test / (n_train + n_test), shuffle=True, random_state=1)

def get_accuracy(true_maps: tf.Tensor, my_maps: tf.Tensor) -> float:
  values = tf.equal(tf.argmax(true_maps, axis=1), tf.argmax(my_maps, axis=1))
  return tf.reduce_sum(tf.cast(values, tf.float64)) / tf.cast(len(true_maps), tf.float64)

network = keras.Sequential([
    layers.Flatten(),
    layers.Dense(4, activation=keras.activations.softsign),
    layers.Dense(4, activation=keras.activations.swish),
    layers.Dense(6, activation=keras.activations.selu, kernel_initializer='lecun_normal'),
    layers.Dense(4, activation=keras.activations.selu),
    layers.Dense(100, activation=keras.activations.softmax),
])

network.build((None, 2))
network.summary()
network.compile(optimizer= keras.optimizers.Adam(learning_rate=1e-3),loss= keras.losses.CategoricalCrossentropy(),metrics = ['accuracy', get_accuracy])

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 2)                 0         
                                                                 
 dense (Dense)               (None, 4)                 12        
                                                                 
 dense_1 (Dense)             (None, 4)                 20        
                                                                 
 dense_2 (Dense)             (None, 6)                 30        
                                                                 
 dense_3 (Dense)             (None, 4)                 28        
                                                                 
 dense_4 (Dense)             (None, 100)               500       
                                                                 
Total params: 590
Trainable params: 590
Non-trainable pa

In [ ]:
history = network.fit(
    x = x_train,
    y = y_train,
    epochs = 100,
    batch_size = batch_size,
    validation_data = (x_test, y_test),
    callbacks = [keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)]
)

score, _, acc = network.evaluate(x_test, y_test, batch_size=batch_size)
print('score: ', score)
print('accuracy: ' + str(acc*100) + '%')

Epoch 1/100
3907/3907 [==============================] - 25s 6ms/step - loss: 1.5231 - accuracy: 0.5834 - get_accuracy: 0.5834 - val_loss: 0.5803 - val_accuracy: 0.8148 - val_get_accuracy: 0.8149
Epoch 2/100
3907/3907 [==============================] - 25s 7ms/step - loss: 0.4504 - accuracy: 0.8573 - get_accuracy: 0.8573 - val_loss: 0.3647 - val_accuracy: 0.8795 - val_get_accuracy: 0.8793
Epoch 3/100
3907/3907 [==============================] - 25s 6ms/step - loss: 0.3204 - accuracy: 0.8964 - get_accuracy: 0.8964 - val_loss: 0.2902 - val_accuracy: 0.9032 - val_get_accuracy: 0.9034
Epoch 4/100
3907/3907 [==============================] - 26s 7ms/step - loss: 0.2676 - accuracy: 0.9123 - get_accuracy: 0.9123 - val_loss: 0.2511 - val_accuracy: 0.9140 - val_get_accuracy: 0.9138
Epoch 5/100
3907/3907 [==============================] - 24s 6ms/step - loss: 0.2357 - accuracy: 0.9221 - get_accuracy: 0.9221 - val_loss: 0.2222 - val_accuracy: 0.9283 - val_get_accuracy: 0.9284
Epoch 6/100
3907/390

In [ ]:
network.save('polar_express_model')
network.summary()

generator = polar_generator(20000, grid=(g1,g2), noise=0.002, flat=True)
accumulators = 0.0
lowerNumber = 0
count = 0
iterations = 1000
while count < iterations:
  (theta, rho),y = next(generator)
  x = np.array(list(zip(theta, rho)))
  score, _, accumulate = network.evaluate(x, y, batch_size=batch_size)
  accumulators += accumulate
  if accumulate < 0.95:
    lowerNumber += 1
  count += 1

print('accuracy: ' + str(accumulators / iterations * 100) + '%')
print('lower 95%: ' + str(lowerNumber) + ' / ' + str(iterations))

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 2)                 0         
                                                                 
 dense (Dense)               (None, 4)                 12        
                                                                 
 dense_1 (Dense)             (None, 4)                 20        
                                                                 
 dense_2 (Dense)             (None, 6)                 30        
                                                                 
 dense_3 (Dense)             (None, 4)                 28        
                                                                 
 dense_4 (Dense)             (None, 100)               500       
                                                                 
Total params: 590
Trainable params: 590
Non-trainable pa

In [ ]:
network.save('polar_express_model_erik3')
!zip -r polar_express_model_erik3.zip polar_express_model_erik3/

  adding: polar_express_model_erik3/ (stored 0%)
  adding: polar_express_model_erik3/variables/ (stored 0%)
  adding: polar_express_model_erik3/variables/variables.data-00000-of-00001 (deflated 47%)
  adding: polar_express_model_erik3/variables/variables.index (deflated 68%)
  adding: polar_express_model_erik3/keras_metadata.pb (deflated 89%)
  adding: polar_express_model_erik3/saved_model.pb (deflated 89%)
  adding: polar_express_model_erik3/assets/ (stored 0%)


Utile esercizio: aggiungete rumore al generatore e verificate l'effetto sulla
"ground truth".

# Cosa consegnare

Ai fini del progetto dovete lavorare con la **griglia di default di dimensione 10x10, e con il rumore di default .002**

il generatore deve essere trattato come una scatola nera: non modificatelo e non sfruttate la sua semantica, che si suppone ignota. Potete lavorare in modlaità "flat", se preferite.

Dovete:

1.   definire una funzione per il calcolo della accuratezza (potete prendere ispirazione dal cocice della cella precedente) 
2.   definire una rete neurale che prende in input theta e rho e restituisce out
3.  misurare l'accuratezza della rete, che deve essere maggiore o uguale del 95%; l'accuratezza deve essere misurata su almeno 20000 dati
4. perfezionare il modello cercando di diminuire il più possibile il numero dei parametri mantenendo una accuratezza superiore al 95%. Solo la vostra rete migliore deve essere consegnata.

Dovete consegnare un UNICO notebook eseguibile su colab, che contenga il codice della rete, il suo sommario con il numero dei parametri, la storia di training, il codice per il calcolo della accuratezza e la sua valutazione sulla vostra rete.

**N.B.** L'accuratezza deve essere superiore o uguale a 95%, ma non influisce in altro modo sulla valutazione. Il vostro punteggio dipenderà unicamente dal numero dei parametri: più è piccolo e più la vostra vaalutazione sarà elevata.  

#Buon lavoro!



